In [2]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [13]:
TRAIN_DIR = '/Users/atharvavirkar/Desktop/DA/Sem-3/Computer Vision/Project/images/train'
TEST_DIR = '/Users/atharvavirkar/Desktop/DA/Sem-3/Computer Vision/Project/images/validation'

In [14]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels


In [15]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

happy completed
sad completed
fear completed
surprise completed
neutral completed
angry completed
disgust completed


In [16]:
print(train)

                                                   image    label
0      /Users/atharvavirkar/Desktop/DA/Sem-3/Computer...    happy
1      /Users/atharvavirkar/Desktop/DA/Sem-3/Computer...    happy
2      /Users/atharvavirkar/Desktop/DA/Sem-3/Computer...    happy
3      /Users/atharvavirkar/Desktop/DA/Sem-3/Computer...    happy
4      /Users/atharvavirkar/Desktop/DA/Sem-3/Computer...    happy
...                                                  ...      ...
28816  /Users/atharvavirkar/Desktop/DA/Sem-3/Computer...  disgust
28817  /Users/atharvavirkar/Desktop/DA/Sem-3/Computer...  disgust
28818  /Users/atharvavirkar/Desktop/DA/Sem-3/Computer...  disgust
28819  /Users/atharvavirkar/Desktop/DA/Sem-3/Computer...  disgust
28820  /Users/atharvavirkar/Desktop/DA/Sem-3/Computer...  disgust

[28821 rows x 2 columns]


In [17]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

happy completed
sad completed
fear completed
surprise completed
neutral completed
angry completed
disgust completed


In [18]:
print(test)
print(test['image'])

                                                  image    label
0     /Users/atharvavirkar/Desktop/DA/Sem-3/Computer...    happy
1     /Users/atharvavirkar/Desktop/DA/Sem-3/Computer...    happy
2     /Users/atharvavirkar/Desktop/DA/Sem-3/Computer...    happy
3     /Users/atharvavirkar/Desktop/DA/Sem-3/Computer...    happy
4     /Users/atharvavirkar/Desktop/DA/Sem-3/Computer...    happy
...                                                 ...      ...
7061  /Users/atharvavirkar/Desktop/DA/Sem-3/Computer...  disgust
7062  /Users/atharvavirkar/Desktop/DA/Sem-3/Computer...  disgust
7063  /Users/atharvavirkar/Desktop/DA/Sem-3/Computer...  disgust
7064  /Users/atharvavirkar/Desktop/DA/Sem-3/Computer...  disgust
7065  /Users/atharvavirkar/Desktop/DA/Sem-3/Computer...  disgust

[7066 rows x 2 columns]
0       /Users/atharvavirkar/Desktop/DA/Sem-3/Computer...
1       /Users/atharvavirkar/Desktop/DA/Sem-3/Computer...
2       /Users/atharvavirkar/Desktop/DA/Sem-3/Computer...
3       /Users/atharv

In [19]:
from tqdm.notebook import tqdm

In [22]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,grayscale =  True )
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features
    

In [23]:
train_features = extract_features(train['image']) 

  0%|          | 0/28821 [00:00<?, ?it/s]

/opt/homebrew/lib/python3.10/site-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [24]:
test_features = extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [25]:
x_train = train_features/255.0
x_test = test_features/255.0

In [26]:
from sklearn.preprocessing import LabelEncoder

In [27]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [28]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [29]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [32]:
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))

In [37]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [38]:
model.fit(x= x_train,y = y_train, batch_size = 128, epochs = 100, validation_data = (x_test,y_test)) 

Epoch 1/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 198s 874ms/step - accuracy: 0.2352 - loss: 1.8391 - val_accuracy: 0.2583 - val_loss: 1.8051
Epoch 2/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 244s 1s/step - accuracy: 0.2505 - loss: 1.8101 - val_accuracy: 0.2455 - val_loss: 1.7925
Epoch 3/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 242s 1s/step - accuracy: 0.2634 - loss: 1.7618 - val_accuracy: 0.3429 - val_loss: 1.6430
Epoch 4/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 237s 1s/step - accuracy: 0.3432 - loss: 1.6354 - val_accuracy: 0.4384 - val_loss: 1.4535
Epoch 5/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 271s 1s/step - accuracy: 0.4036 - loss: 1.5258 - val_accuracy: 0.4674 - val_loss: 1.4007
Epoch 6/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 336s 1s/step - accuracy: 0.4380 - loss: 1.4510 - val_accuracy: 0.4984 - val_loss: 1.3087
Epoch 7/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 399s 2s/step - accuracy: 0.4534 - loss: 1.4151 - val_accuracy: 0.5052 - val_loss: 1.2872
Epoch 8/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 499s 2s/step - accuracy: 0.4747 - loss: 1.3748

In [42]:
model_json = model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save('model.keras')

In [43]:
from keras.models import model_from_json

In [47]:
import os
print(os.getcwd())

/Users/atharvavirkar/Desktop/DA/Sem-3/Computer Vision/Project/Face_Emotion_Recognition_Machine_Learning-main


In [51]:
# json_file = open("facialemotionmodel.json", "r")
# model_json = json_file.read()
# json_file.close()
# model = model_from_json(model_json)
# model.load_weights("facialemotionmodel.h5")

In [52]:
label = ['angry','disgust','fear','happy','neutral','sad','surprise']

In [55]:
def ef(image):
    img = load_img(image,color_mode="grayscale" )
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0
    

In [65]:
image = '/Users/atharvavirkar/Desktop/DA/Sem-3/Computer Vision/Project/images/validation/sad/1003.jpg'
print("original image is of sad")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)

original image is of sad
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
model prediction is  sad


In [74]:
# import matplotlib.pyplot as plt
# %matplotlib inline

In [80]:
image = '/Users/atharvavirkar/Desktop/DA/Sem-3/Computer Vision/Project/images/validation/sad/1003.jpg'
print("original image is of sad")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
# plt.imshow(img.reshape(48,48),cmap='gray')

original image is of sad
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
model prediction is  sad


In [81]:
image = '/Users/atharvavirkar/Desktop/DA/Sem-3/Computer Vision/Project/images/validation/fear/21.jpg'
print("original image is of fear")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
# plt.imshow(img.reshape(48,48),cmap='gray')

original image is of fear
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
model prediction is  fear


In [83]:
image = '/Users/atharvavirkar/Desktop/DA/Sem-3/Computer Vision/Project/images/validation/disgust/573.jpg'
print("original image is of disgust")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
# plt.imshow(img.reshape(48,48),cmap='gray')

original image is of disgust
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
model prediction is  disgust


In [85]:
image = '/Users/atharvavirkar/Desktop/DA/Sem-3/Computer Vision/Project/images/validation/happy/8.jpg'
print("original image is of happy")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
# plt.imshow(img.reshape(48,48),cmap='gray')

original image is of happy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
model prediction is  happy


In [87]:
image = '/Users/atharvavirkar/Desktop/DA/Sem-3/Computer Vision/Project/images/validation/surprise/78.jpg'
print("original image is of surprise")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
# plt.imshow(img.reshape(48,48),cmap='gray')

original image is of surprise
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
model prediction is  surprise
